In [ ]:
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor

train = FashionMNIST(root='data',
                     train=True,
                     transform=ToTensor(),
                     download=True)
test = FashionMNIST(root='data',
                    train=False,
                    transform=ToTensor(),
                    download=True)

In [8]:
# making a class that provides data of 2 classes
class DataProvider:
  def __init__(self, train, test) -> None:
    self.train = train
    self.test = test

  def getTwoClasses(self, first: int, second: int):
    first_train_mask = self.train.targets == first
    first_train_data = self.train.data[first_train_mask]

    second_train_mask = self.train.targets == second
    second_train_data = self.train.data[second_train_mask]

    first_test_mask = self.test.targets == first
    first_test_data = self.test.data[first_test_mask]

    second_test_mask = self.test.targets == second
    second_test_data = self.test.data[second_test_mask]
    return first_train_data, second_train_data, first_test_data, second_test_data

In [9]:
dp = DataProvider(train, test)

In [11]:
dress_train, coat_train, dress_test, coat_test = dp.getTwoClasses(3, 4)

In [1]:
import torch
import torch.nn as nn

In [27]:
conv0 = nn.Conv2d(1, 16, 3, 1)
sample_x = dress_train[0].reshape(-1, 1, 28, 28) / 255.0
print(sample_x.shape)
y0 = conv0(sample_x)
print(y0.shape)
relu0 = nn.ReLU()
y1 = relu0(y0)
print(y1.shape)
max0 = nn.MaxPool2d(2)
y2 = max0(y1)
print(y2.shape)

torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 26, 26])
torch.Size([1, 16, 26, 26])
torch.Size([1, 16, 13, 13])


In [40]:
class MyNet(nn.Module):
  def __init__(self, in_width=28, in_height=28, in_channels=1) -> None:
    super().__init__()
    self.nn = nn.Sequential(
      nn.Conv2d(in_channels, 16, 3),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Conv2d(16, 32, 3),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Conv2d(32, 64, 3),
      nn.Flatten(),
      nn.Linear(64*3*3, 16),
      nn.ReLU(),
      nn.Linear(16, 2),
      nn.Softmax(dim=1)
    )

  def forward(self, x):
    return self.nn(x)

In [41]:
model = MyNet()
print(model)

MyNet(
  (nn): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=576, out_features=16, bias=True)
    (9): ReLU()
    (10): Linear(in_features=16, out_features=2, bias=True)
    (11): Softmax(dim=1)
  )
)


In [44]:
from torch.utils.data import Dataset

In [54]:
class MyDataset(Dataset):
  def __init__(self, train_1, train_2) -> None:
    t1 = train_1.reshape(-1, 1, 28, 28) / 255.0
    t2 = train_2.reshape(-1, 1, 28, 28) / 255.0
    l1 = torch.zeros(t1.shape[0], dtype=torch.int64)
    l2 = torch.zeros(t2.shape[0], dtype=torch.int64)
    t = torch.cat((t1, t2), 0)
    l = torch.cat((l1, l2), 0)
    r = torch.randperm(len(l))
    self.data = t[r]
    self.target = l[r]

  def __len__(self):
    return len(self.target)

  def __getitem__(self, index):
    return self.data[index], self.target[index]

In [68]:
train_dataset = MyDataset(dress_train[:250], coat_train[:250])
test_dataset = MyDataset(dress_test[:100], coat_test[:100])

In [69]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [70]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model = MyNet().to(device)
print(model)

Using cuda device
MyNet(
  (nn): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=576, out_features=16, bias=True)
    (9): ReLU()
    (10): Linear(in_features=16, out_features=2, bias=True)
    (11): Softmax(dim=1)
  )
)


In [71]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [72]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [73]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [74]:
epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.612819  [    0/  500]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.605137 

Epoch 2
-------------------------------
loss: 0.605735  [    0/  500]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.598447 

Epoch 3
-------------------------------
loss: 0.598455  [    0/  500]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.592689 

Epoch 4
-------------------------------
loss: 0.592937  [    0/  500]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.587696 

Epoch 5
-------------------------------
loss: 0.588161  [    0/  500]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.582919 

Epoch 6
-------------------------------
loss: 0.584019  [    0/  500]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.578231 

Epoch 7
-------------------------------
loss: 0.578673  [    0/  500]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.573681 

Epoch 8
-------------------------------
loss: 0.572535  [    0/  500]
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.569074 

Epoch 9
--------

In [ ]:
class MyAE(nn.Module):
  def __init__(self, in_width=28, in_height=28, in_channels=1) -> None:
    super().__init__()

    self.encoder = nn.Sequential(
      nn.Conv2d(in_channels=in_channels,
                out_channels=16,
                kernel_size=3)
      
    )